<a href="https://colab.research.google.com/github/basantyoussry/AI-Powered-Recipe-Assistant-/blob/main/Recipe_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [](http://)**Requiermnets**

* langchain
* huggingface
* Transformers
* Spoonacular
* gtts
* Diffusers
* requests
* moviepy


In [ ]:
!pip install spoonacular
!pip install gtts

In [ ]:
import transformers
import spoonacular as sp
import requests
from diffusers import DiffusionPipeline
import torch
from tqdm import tqdm  # Import tqdm for progress bar
from gtts import gTTS


from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip


In [ ]:


# Spoonacular API endpoints
search_url = "https://api.spoonacular.com/recipes/complexSearch"
recipe_info_url = "https://api.spoonacular.com/recipes/{id}/information"

# API key (replace with your own)
api_key = "72c91728462848de9fb3720585de6e02"

def fetch_recipe_instructions(query="Cookies", num_results=1):
    """
    Fetches recipe instructions from the Spoonacular API.
    """
    # Parameters for the search
    params = {
        "query": query,  # Search query
        "number": num_results,  # Number of results to return
        "instructionRequired": True,
        "addRecipeInformation": True,
        "fillIngredients": True,
        "apiKey": api_key
    }

    # Make the search request
    response = requests.get(search_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        recipes = data.get("results", [])

        if not recipes:
            print("No recipes found. Try a different search query.")
            return None

        # Loop through each recipe
        for recipe in recipes:
            print(f"\nRecipe: {recipe['title']}")
            print(f"ID: {recipe['id']}")
            print(f"Image: {recipe['image']}")

            # Fetch detailed recipe information using the recipe ID
            recipe_id = recipe['id']
            info_response = requests.get(recipe_info_url.format(id=recipe_id), params={"apiKey": api_key})

            if info_response.status_code == 200:
                recipe_details = info_response.json()

                # Print ingredients
                print("\nIngredients:")
                for ingredient in recipe_details.get("extendedIngredients", []):
                    print(f"- {ingredient['original']}")

                # Print instructions
                print("\nInstructions:")
                if "instructions" in recipe_details and recipe_details["instructions"]:
                    instructions = recipe_details["instructions"]
                    print(instructions)
                    return instructions  # Return the instructions for Stable Diffusion
                else:
                    print("No instructions available.")
                    return None
            else:
                print(f"Error fetching details for recipe ID {recipe_id}: {info_response.status_code}")
                return None
    else:
        print(f"Error: {response.status_code}")
        return None

# Fetch instructions from Spoonacular API
instructions = fetch_recipe_instructions()
if instructions:
    print("\nInstructions fetched successfully!")
else:
    print("\nFailed to fetch instructions.")


Recipe: Oreo Mini Cheesecake
ID: 654034
Image: https://img.spoonacular.com/recipes/654034-312x231.jpg

Ingredients:
- 250g Cream cheese
- 150g Whipping cream
- 1 tsp Vanilla extract
- 2 Eggs (70g each), separated
- a pinch of cream of tartar
- 60g Sugar
- 1 tube of Oreo (137g)

Instructions:
Separate Oreos, leaving 12 pcs without cream intact and round. Put the rest of the biscuits together with the filling into a food processor and pulse in fine.
Remember to preheat your oven (150C) and boil water after this.
Line big muffin tin (100ml capacity each) with 12 paper cups. Place 1 heaped teaspoon of oreo crumbs onto paper cups and  lightly press the crumbs to flatten with a mini roller pin. (see pictures)
Cream cream cheese  and 30g sugar on low speed until smooth, put in whipping cream, vanilla extract and egg yolks and continue to beat on low speed for another minute until well combined. Transfer cream mixtures to a large bowl and set aside.
Whisk egg whites until frothy, add in cream

In [ ]:
import re
from diffusers import DiffusionPipeline
from tqdm import tqdm

# Remove HTML tags using regex
instructions = re.sub(r"<.*?>", "", instructions)

# Split instructions into steps
steps = [step.strip() for step in instructions.split("\n") if step.strip()]
print("\nProcessed steps:")
for i, step in enumerate(steps):
  print(f"Step {i+1}: {step}")


def preprocess_instructions(instructions):
    """
    Preprocesses instructions by removing HTML tags and splitting long steps.
    """


    # Ensure each step is within the token limit (77 tokens)
    processed_steps = []
    for step in steps:
        if len(step.split()) <= 77:  # Check token length (approximate)
            processed_steps.append(step)
        else:
            # Split long steps into smaller chunks
            words = step.split()
            for i in range(0, len(words), 77):
                chunk = " ".join(words[i:i+77])
                processed_steps.append(chunk)

    return processed_steps

def generate_images_from_instructions(instructions):
    """
    Generates images for each step of the instructions using Stable Diffusion.
    """
    if not instructions:
        print("No instructions provided. Skipping image generation.")
        return

    # Load the Stable Diffusion pipeline
    pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    )
    pipe.to("cuda")  # Ensure this is uncommented if you have a GPU

    # Generate images for each step with a progress bar
    for i, step in enumerate(tqdm(steps, desc="Generating images")):
        try:
            # Generate an image for the current step
            output = pipe(prompt=step).images[0]  # Generate one image per step
            output.save(f"step_{i+1}.png")  # Save the image with a step-specific filename
            print(f"Saved image for step {i+1}: {step}")
        except Exception as e:
            print(f"Error generating image for step {i+1}: {e}")



# Generate images using Stable Diffusion
if instructions:
    generate_images_from_instructions(instructions)
else:
    print("No instructions available for image generation.")


Processed steps:
Step 1: Separate Oreos, leaving 12 pcs without cream intact and round. Put the rest of the biscuits together with the filling into a food processor and pulse in fine.
Step 2: Remember to preheat your oven (150C) and boil water after this.
Step 3: Line big muffin tin (100ml capacity each) with 12 paper cups. Place 1 heaped teaspoon of oreo crumbs onto paper cups and  lightly press the crumbs to flatten with a mini roller pin. (see pictures)
Step 4: Cream cream cheese  and 30g sugar on low speed until smooth, put in whipping cream, vanilla extract and egg yolks and continue to beat on low speed for another minute until well combined. Transfer cream mixtures to a large bowl and set aside.
Step 5: Whisk egg whites until frothy, add in cream of tartar continue to beat until soft peaks. Add in 30g sugar and beat till stiff (as long as egg whites don't fall when overturned, it will be fine, no need to be pointy stiff).
Step 6: Fold in egg whites to cream cheese mixtures with

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Generating images:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Generating images:  10%|█         | 1/10 [00:48<07:20, 48.94s/it]

...

Generating images:  90%|█████████ | 9/10 [08:08<00:54, 54.98s/it]

Saved image for step 9: Leave to cool in oven, door ajar. Cool completely before chilling.


  0%|          | 0/50 [00:00<?, ?it/s]

Generating images: 100%|██████████| 10/10 [09:03<00:00, 54.35s/it]

Saved image for step 10: Best chilled for 3 hours or more before serving.


In [ ]:

# Generate voiceover for a step
def generate_voiceover(text, output_file):
    tts = gTTS(text=text, lang="en", slow=False)
    tts.save(output_file)

# Generate voiceovers for each step
for i, sinstruction in enumerate(steps):
    output_file = f"step_{i+1}.mp3"
    generate_voiceover(step, output_file)
    print(f"Generated voiceover for: {step}\nSaved to: {output_file}")

Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_1.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_2.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_3.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_4.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_5.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_6.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_7.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_8.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_9.mp3
Generated voiceover for: Best chilled for 3 hours or more before serving.
Saved to: step_10.mp3


In [ ]:
def create_video_from_steps(steps):
    """
    Creates a video from the generated images and voiceovers.
    """
    clips = []
    for i, step in enumerate(steps):
        # Load the image
        image_path = f"step_{i+1}.png"
        image_clip = ImageClip(image_path).set_duration(5)  # 5 seconds per step

        # Load the voiceover
        voiceover_path = f"step_{i+1}.mp3"
        audio_clip = AudioFileClip(voiceover_path)
        image_clip = image_clip.set_audio(audio_clip)

        clips.append(image_clip)

    # Combine all clips into a single video
    final_video = concatenate_videoclips(clips)
    final_video.write_videofile("recipe_video.mp4", fps=24)
    print("Video created successfully: recipe_video.mp4")
